## 2.1注意力机制

文本中有点一段话非常妙 “ 注意力机制的本质是对两对序列的元素一次进行相似度计算，寻找出一个序列
的每个元素对另一个序列的每个元素的相关度，然后基于相关度进行加权，即分配注意力”

而我理解的自注意力也是这样，通过把词转换为维度为d的词向量，之后以这个词向量与不同的权重矩阵
W^Q,W^K,W^V(简化为均为d×d)进行矩阵乘法得到q,k,v向量的值，接下来就是就是注意力的计算：
对于当前词向量产生的q，即query，它需要与所有词语进行相互交流（或者是相似度的计算），也就是询问，
对应的计算为q× k.transpose()，得到n × n的矩阵（假设序列的长度为n），对于其中的第 i 行，
它的含义不就是第 i 个词对于所有 n 个词进行的相似度计算嘛，所以一开始主对角线上的值相比于其他位置
一定是很大的，之后经过softmax(dim=-1) 后，得到的 n × n矩阵就是V的概率权重矩阵，之后对于每个词的v值
进行加权求和。
下面是我理解的计算过程




![](../imgae/未命名图片.png)


inputs.size = (B,T)    B 为batch_size的维度，T为序列的长度
之后经过查词向量表 得到 
input.size = (B,T,C)其中C为向量的维度
它与Q,K,V（C × C）的权重矩阵相乘得到 整个输入的q,k,v值（B，T，C）
之后就是q × k.transpose(-2,-1) 得到 （B,T,T）
经过softmax(dim=-1) 之后得到概率分布
然后(B,T,T) 与 v值相乘 得到(B,T,C) 这时所有的词语之间进行了一定程度上的信息交流


多头注意力 我的理解是 把 （B，T，C）改为了 （B，T，n_head,C// n_head）

In [ ]:
# 参考代码
import torch 
import torch.nn as nn
import math
class MultiHeadAttention(nn.Module):
    def __init__(self,n_embd,n_head,attn_pdrop=0.1,resid_pdrop=0.1):
        super(MultiHeadAttention,self).__init__()
        assert n_embd % n_head == 0 

        self.c_attn = nn.Linear(n_embd,3*n_embd)
        self.c_proj = nn.Linear(n_embd,n_embd)

        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)

        
        self.n_head = n_head
        self.n_embd = n_embd
    def forward(self,x,mask=None):
        B,T,C = x.size()
        q,k,v = self.c_attn(x).split(self.n_embd,dim=-1)
        k = k.view(B,T,self.n_head,C// self.n_head).transpose(1,2)
        q = q.view(B,T,self.n_head,C// self.n_head).transpose(1,2)
        v = v.view(B,T,self.n_head,C// self.n_head).transpose(1,2)

        mask = mask.view(1,1,T,T)

        attn = (q @ k.transpose(-2,-1))*(1.0 /math.sqrt(k.size(-1)) )

        attn.masked_fill_(mask,float('-inf'))  #  decoder 和 encoder 的掩码形式是不同的

        attn = torch.softmax(attn,dim=-1)
        attn = self.attn_drop(attn)
        y = attn@v
        y= y.transpose(1,2).contiguous().view(B,T,C)
        y = self.resid_drop(self.c_proj(y))
        return y
    

#  自回归时的示例用法
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#mask = torch.triu(torch.ones(T,T,device = device),diagonal=1).bool()
#mask = mask.view(1,1,T,T)